In [0]:
from pyspark.sql import functions as F
df=spark.table("default.sw_indices_merged")
initial_value=df.orderBy("Date").limit(1).collect()[0]
N50c=initial_value["Nifty_Close"]
SEc=initial_value["SwarajEng_Close"]
Autoc=initial_value["CNXAUTO_Close"]
df=df.withColumn("Nifty_Return",F.round((F.col("Nifty_Close")-F.lit(N50c))*100.00/F.lit(N50c),2)).withColumn("SwarajEng_Return",F.round((F.col("SwarajEng_Close")-F.lit(SEc))*100.00/SEc,2)).withColumn("CNXAUTO_Return",F.round((F.col("CNXAUTO_Close")-F.lit(Autoc))*100.00/F.lit(Autoc),2))
df.write.option("overwriteSchema","true").mode("overwrite").saveAsTable("sw_indices_returns")

In [0]:
df.show(10)

In [0]:
%sql
-- Creating view for partition creation of a table
create or replace view Indices_temp
as 
select *,year(Date) as year,month(Date) as month
from sw_indices_returns;

In [0]:
%sql
-- Creating a partition table
create or replace table Indices
using delta
partitioned by (year,month)
as 
select * from Indices_temp;

In [0]:
%sql

-- Drop staging table
--drop table if exists default.sw_indices_merged;

-- Drop table after the load
drop view if exists Indices_temp;

drop table if exists sw_indices_returns;

In [0]:
%sql
select Date,Nifty_Return,SwarajEng_Return,CNXAUTO_Return 
from Indices
order by Date desc
limit 10;

In [0]:
spark.sql("SHOW TABLES IN default").show()

In [0]:
spark.catalog.listTables("default")
